## **Importação de dados**

In [0]:
# Instalar bibliotecas necessárias
%pip install requests tqdm pandas


In [0]:
# Importações
import os
import requests
import tarfile
from pyspark.sql.functions import lit
import pandas as pd
from tqdm import tqdm
from pyspark.sql.functions import col, to_date, to_timestamp
from pyspark.sql.types import StringType, BooleanType, IntegerType, DoubleType
from pyspark.sql.functions import avg, sum, countDistinct, stddev
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pyspark.sql import functions as F
from scipy.stats import ttest_ind
import matplotlib.ticker as mtick
import math
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.window import Window

In [0]:
# Função para baixar arquivos diretamente para /dbfs/ (Databricks File System)
def baixar_arquivos(urls, dbfs_dir="/dbfs/FileStore/iFood/dataset"):
    """
    Faz download dos arquivos das URLs e salva no diretório DBFS especificado.
    """
    os.makedirs(dbfs_dir, exist_ok=True)

    for url in urls:
        nome_arquivo = url.split("/")[-1]
        local_path = os.path.join(dbfs_dir, nome_arquivo)
        if os.path.exists(local_path):
            print(f"Arquivo já existe: {local_path}")
            continue

        print(f"Baixando: {url}")
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(local_path, 'wb') as f:
                for chunk in tqdm(response.iter_content(chunk_size=8192), desc=f"⬇️ {nome_arquivo}"):
                    f.write(chunk)
            print(f"Salvo em: {local_path}\n")
        else:
            print(f"Erro ao baixar {url} - Status code: {response.status_code}")



In [0]:
# URLs para download
urls = [
    "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz",
    "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz",
    "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz",
    "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
]


In [0]:
# Baixar arquivos
baixar_arquivos(urls, "/dbfs/FileStore/iFood/dataset")

In [0]:
# Função para extrair arquivos .tar.gz do DBFS para diretório no DBFS
def extract_all_files(tar_file_path, extract_to):
    """
    Extrai o conteúdo de um arquivo tar.gz localizado no DBFS para um diretório também no DBFS.
    """
    local_tar_path = "/tmp/temp_file.tar.gz"
    local_extract_path = "/tmp/extracted/"

    # Copiar arquivo do DBFS para local temporário
    if not os.path.exists(local_tar_path):
        os.system(f"cp {tar_file_path} {local_tar_path}")

    # Criar diretório de extração temporário
    if not os.path.exists(local_extract_path):
        os.makedirs(local_extract_path)

    # Extrair o tar.gz
    with tarfile.open(local_tar_path, 'r:gz') as tar:
        tar.extractall(path=local_extract_path)

    # Copiar os arquivos extraídos para o DBFS
    os.makedirs(extract_to, exist_ok=True)
    for root, dirs, files in os.walk(local_extract_path):
        for file in files:
            src_file = os.path.join(root, file)
            dst_file = os.path.join(extract_to, file)
            os.system(f"cp {src_file} {dst_file}")

    print(f"Arquivos extraídos para: {extract_to}")

# Executar extração
extract_all_files(
    tar_file_path='/dbfs/FileStore/iFood/dataset/ab_test_ref.tar.gz', 
    extract_to='/dbfs/FileStore/iFood/dataset/extracted'
)


In [0]:
# Leitura dos arquivos
df_users = spark.read.option("compression", "gzip").csv("/FileStore/iFood/dataset/consumer.csv.gz", header=True, inferSchema=True)
df_restaurants = spark.read.option("compression", "gzip").csv("/FileStore/iFood/dataset/restaurant.csv.gz", header=True, inferSchema=True)
df_orders = spark.read.option("compression", "gzip").json("/FileStore/iFood/dataset/order.json.gz")
df_abtest = spark.read.options(header=True, inferSchema=True).csv('/FileStore/iFood/dataset/extracted/ab_test_ref.csv')

In [0]:
#Juntar (join) com o DataFrame de pedidos para analisar comportamento

df_abtest_orders = df_orders.join(df_abtest, on="customer_id", how="inner")


### Segmentação Avançada — Receita e Comportamento



In [0]:
# ===============================
# 🧠 1. Calcular métricas por cliente
# ===============================

# 🔸 Calcular frequência, ticket médio e ARPU
df_clientes = (
    df_abtest_orders
    .groupBy("customer_id")
    .agg(
        F.countDistinct("order_id").alias("frequencia_pedidos"),
        F.sum("order_total_amount").alias("receita_total"),
        (F.sum("order_total_amount") / F.countDistinct("order_id")).alias("ticket_medio")
    )
    .withColumn("arpu", F.col("receita_total"))  # ARPU = receita_total (já que é receita por cliente no período)
)

# ===============================
# 📏 2. Calcular os percentis (25 e 75)
# ===============================

# 🔸 Coletar os percentis
percentis = df_clientes.selectExpr(
    "percentile_approx(arpu, array(0.25, 0.75)) as arpu_percentis",
    "percentile_approx(frequencia_pedidos, array(0.25, 0.75)) as freq_percentis",
    "percentile_approx(ticket_medio, array(0.25, 0.75)) as ticket_percentis"
).collect()[0]

# 🔍 Extrair os valores
arpu_p25, arpu_p75 = percentis["arpu_percentis"]
freq_p25, freq_p75 = percentis["freq_percentis"]
ticket_p25, ticket_p75 = percentis["ticket_percentis"]

# ===============================
# 🏷️ 3. Criar a coluna de Segmento
# ===============================

df_segmentado = (
    df_clientes
    .withColumn(
        "segmento",
        F.when(
            (F.col("arpu") >= arpu_p75) |
            ((F.col("ticket_medio") >= ticket_p75) & (F.col("frequencia_pedidos") >= freq_p75)),
            "Alta Receita 🔴"
        ).when(
            (F.col("arpu") <= arpu_p25) |
            ((F.col("frequencia_pedidos") <= freq_p25) & (F.col("ticket_medio") <= ticket_p25)),
            "Baixa Receita ⚪"
        ).otherwise("Média Receita ⚫")
    )
)

# ▶️ Visualizar a tabela segmentada
df_segmentado.display()

# ===============================
# 📊 4. Distribuição dos Segmentos
# ===============================

# 🔸 Contagem dos clientes por segmento
segmento_count = (
    df_segmentado
    .groupBy("segmento")
    .agg(F.countDistinct("customer_id").alias("quantidade_clientes"))
    .orderBy(F.col("quantidade_clientes").desc())
)

# 🔄 Converter para Pandas
segmento_count_pd = segmento_count.toPandas()

# 🎨 Definir cores dos segmentos
colors = ['#EA1D2C' if s == 'Alta Receita 🔴' 
          else '#2E2E2E' if s == 'Média Receita ⚫' 
          else '#CCCCCC' for s in segmento_count_pd['segmento']]

# 📈 Plot do gráfico de barras
plt.figure(figsize=(7,5))
bars = plt.bar(
    segmento_count_pd['segmento'],
    segmento_count_pd['quantidade_clientes'],
    color=colors
)

# 🏷️ Adicionar rótulos nos bares
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f'{int(height):,}'.replace(',', '.'),
        ha='center', va='bottom',
        fontsize=10, fontweight='bold', color='#333333'
    )

plt.xlabel('Segmento', fontsize=10, fontweight='bold')
plt.ylabel('Quantidade de Clientes', fontsize=10, fontweight='bold')
plt.title('Distribuição de Clientes por Segmento de Receita', fontsize=12, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()



### Analisar os resultados do teste A/B com base nos segmentos novos

In [0]:
# ===============================
# 🔗 1. Juntar pedidos com a segmentação
# ===============================

df_orders_segmentado = (
    df_abtest_orders.join(df_segmentado.select("customer_id", "segmento"), on="customer_id", how="inner")
)

# ===============================
# 📊 2. Calcular métricas por Segmento
# ===============================

metrics_segmento = (
    df_orders_segmentado
    .groupBy("segmento")
    .agg(
        F.countDistinct("customer_id").alias("clientes_unicos"),
        F.countDistinct("order_id").alias("pedidos_unicos"),
        F.sum("order_total_amount").alias("receita_total"),
        (F.sum("order_total_amount") / F.countDistinct("customer_id")).alias("arpu"),
        (F.sum("order_total_amount") / F.countDistinct("order_id")).alias("ticket_medio"),
        (F.countDistinct("order_id") / F.countDistinct("customer_id")).alias("frequencia_media")
    )
    .orderBy(F.col("receita_total").desc())
)

# ▶️ Visualizar
metrics_segmento.display()


In [0]:
# 1️⃣ Agregar dados por cliente


# Calcular frequência, receita total, ticket médio e ARPU
cliente_metrics = df_abtest_orders.groupBy('customer_id').agg(
    F.countDistinct('order_id').alias('frequencia_pedidos'),
    F.sum('order_total_amount').alias('receita_total'),
    (F.sum('order_total_amount') / F.countDistinct('order_id')).alias('ticket_medio')
)

# Calcular ARPU (Receita média por cliente = receita total)
cliente_metrics = cliente_metrics.withColumn('arpu', F.col('receita_total'))

# 2️⃣ Calcular percentis para segmentação
# Obter os percentis de ARPU e Ticket Médio
arpu_percentis = cliente_metrics.approxQuantile('arpu', [0.25, 0.75], 0.01)
ticket_percentis = cliente_metrics.approxQuantile('ticket_medio', [0.25, 0.75], 0.01)

arpu_p25, arpu_p75 = arpu_percentis
ticket_p25, ticket_p75 = ticket_percentis

# 3️⃣ Criar a segmentação
cliente_metrics = cliente_metrics.withColumn(
    'segmento',
    F.when(
        (F.col('arpu') >= arpu_p75) |
        ((F.col('ticket_medio') >= ticket_p75) & (F.col('frequencia_pedidos') >= 3)),
        'Alta Receita 🔴'
    ).when(
        (F.col('arpu') <= arpu_p25) |
        ((F.col('frequencia_pedidos') <= 1) & (F.col('ticket_medio') <= ticket_p25)),
        'Baixa Receita ⚪'
    ).otherwise('Média Receita ⚫')
)

# ✔️ Resultado do dataframe com as métricas e segmentos
cliente_metrics.display()


In [0]:
# ===============================
# 🔄 Converter para Pandas para plotagem
# ===============================

metrics_pd = metrics_segmento.toPandas()

# 🔄 Ordenar os segmentos
order_segmentos = ['Alta Receita 🔴', 'Média Receita ⚫', 'Baixa Receita ⚪']
metrics_pd['segmento'] = pd.Categorical(metrics_pd['segmento'], categories=order_segmentos, ordered=True)
metrics_pd = metrics_pd.sort_values('segmento')

# 🎨 Definir cores dos segmentos
colors = ['#EA1D2C', '#2E2E2E', '#CCCCCC']

# ===============================
# 📈 1. Gráfico de ARPU
# ===============================

plt.figure(figsize=(7,5))
bars = plt.bar(metrics_pd['segmento'], metrics_pd['arpu'], color=colors)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height,
             f'R$ {height:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.'),
             ha='center', va='bottom', fontsize=10, fontweight='bold', color='#333333')

plt.title('ARPU por Segmento', fontsize=12, fontweight='bold')
plt.ylabel('ARPU (R$)', fontsize=10, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# ===============================
# 📈 2. Gráfico de Ticket Médio
# ===============================

plt.figure(figsize=(7,5))
bars = plt.bar(metrics_pd['segmento'], metrics_pd['ticket_medio'], color=colors)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height,
             f'R$ {height:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.'),
             ha='center', va='bottom', fontsize=10, fontweight='bold', color='#333333')

plt.title('Ticket Médio por Segmento', fontsize=12, fontweight='bold')
plt.ylabel('Ticket Médio (R$)', fontsize=10, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# ===============================
# 📈 3. Gráfico de Frequência Média
# ===============================

plt.figure(figsize=(7,5))
bars = plt.bar(metrics_pd['segmento'], metrics_pd['frequencia_media'], color=colors)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height,
             f'{height:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.'),
             ha='center', va='bottom', fontsize=10, fontweight='bold', color='#333333')

plt.title('Frequência Média de Pedidos por Segmento', fontsize=12, fontweight='bold')
plt.ylabel('Frequência Média', fontsize=10, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## Analise financeira

In [0]:
from pyspark.sql import functions as F

# Valor do cupom
valor_cupom = 10

# Agrupar por segmento, calcular receita total e quantidade de clientes distintos
df_financeiro_segmento = (
    df_segmentado
    .groupBy("segmento")
    .agg(
        F.sum("receita_total").alias("receita_total"),
        F.countDistinct("customer_id").alias("clientes")
    )
)

# Converter para pandas para cálculos financeiros
financeiro_pd = df_financeiro_segmento.toPandas()

resultados = []

for _, row in financeiro_pd.iterrows():
    segmento = row['segmento']
    receita_total = row['receita_total']
    clientes = row['clientes']
    
    # Receita Incremental = receita total da campanha (considerando receita total)
    receita_incremental = receita_total
    
    # Custo da campanha = número de clientes * valor do cupom
    custo_campanha = clientes * valor_cupom
    
    # Resultado líquido
    resultado_liquido = receita_incremental - custo_campanha
    
    # ROI (em %)
    roi = (resultado_liquido / custo_campanha) * 100 if custo_campanha != 0 else 0
    
    resultados.append({
        "Segmento": segmento,
        "Receita Total (R$)": receita_total,
        "Clientes": clientes,
        "Receita Incremental (R$)": receita_incremental,
        "Custo da Campanha (R$)": custo_campanha,
        "Resultado Líquido (R$)": resultado_liquido,
        "ROI (%)": roi
    })

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

df_resultado_financeiro = pd.DataFrame(resultados)
display(df_resultado_financeiro)


In [0]:

import matplotlib.pyplot as plt

# Preparar dados para gráfico
segmentos = df_resultado_financeiro["Segmento"]
receita = df_resultado_financeiro["Receita Incremental (R$)"]
custo = df_resultado_financeiro["Custo da Campanha (R$)"]
resultado = df_resultado_financeiro["Resultado Líquido (R$)"]
roi = df_resultado_financeiro["ROI (%)"]

# Cores iFood para segmentos
colors = ['#EA1D2C' if seg == 'Alta Receita 🔴' else '#2E2E2E' if seg == 'Média Receita ⚫' else '#CCCCCC' for seg in segmentos]

plt.figure(figsize=(12, 8))

# Receita vs Custo
plt.subplot(2, 1, 1)
bar_width = 0.35
x = range(len(segmentos))

plt.bar(x, receita, width=bar_width, color=colors, label='Receita Incremental (R$)')
plt.bar([i + bar_width for i in x], custo, width=bar_width, color='#FFD700', label='Custo da Campanha (R$)')

plt.xticks([i + bar_width / 2 for i in x], segmentos, fontsize=12, fontweight='bold')
plt.ylabel('R$ (Reais)', fontsize=12)
plt.title('Receita Incremental e Custo da Campanha por Segmento', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Resultado Líquido e ROI
plt.subplot(2, 1, 2)
plt.bar(x, resultado, width=bar_width, color=colors, label='Resultado Líquido (R$)')
plt.xticks(x, segmentos, fontsize=12, fontweight='bold')
plt.ylabel('R$ (Reais)', fontsize=12)
plt.title('Resultado Líquido por Segmento', fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)

for i, val in enumerate(resultado):
    plt.text(i, val + max(resultado)*0.02, f'{val:,.0f}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Gráfico de ROI (%)
plt.figure(figsize=(8,5))
plt.bar(segmentos, roi, color=colors)
plt.title('ROI (%) por Segmento', fontsize=14, fontweight='bold')
plt.ylabel('ROI (%)', fontsize=12)
plt.xlabel('Segmento', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

for i, val in enumerate(roi):
    plt.text(i, val + max(roi)*0.02, f'{val:.2f}%', ha='center', fontsize=10, fontweight='bold')

plt.show()




In [0]:


# Dados base
clientes_unicos = 806466
receita_total = 24134545.43

# Receita incremental (exemplo, 5% da receita total)
receita_incremental = receita_total * 0.05

# Lista de valores de cupom simulados
valores_cupom = list(range(1, 21))

import matplotlib.pyplot as plt

# Dados
clientes_unicos = 806466
receita_incremental = 19211961.918  # Receita incremental

# Valores de cupom simulados
valores_cupom = list(range(1, 21))

# Listas para armazenar resultados
roi_percentual = []
custos = []
resultados_liquidos = []

# Cálculo
for valor_cupom in valores_cupom:
    custo = valor_cupom * clientes_unicos
    resultado_liquido = receita_incremental - custo
    roi = (resultado_liquido / custo) * 100 if custo != 0 else 0

    custos.append(custo)
    resultados_liquidos.append(resultado_liquido)
    roi_percentual.append(roi)

# 📈 Plotando o gráfico
plt.figure(figsize=(9,5))
plt.plot(valores_cupom, roi_percentual, marker='o', color='#EA1D2C', linewidth=2)
plt.axhline(0, color='gray', linestyle='--')

plt.title('Relação entre Valor do Cupom e ROI (%)', fontsize=14, fontweight='bold')
plt.xlabel('Valor do Cupom (R$)', fontsize=12)
plt.ylabel('ROI (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

# 🔖 Adicionando rótulos nos pontos
for x, y in zip(valores_cupom, roi_percentual):
    plt.text(x, y, f'{y:.1f}%', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

